In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import sqlite3
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.tree import export_text

In [46]:
con = sqlite3.connect("Dataset/database.db")

awards_players_df = pd.read_sql_query("SELECT * FROM awards_players", con)
coaches_df = pd.read_sql_query("SELECT * FROM coaches", con)
players_teams_df = pd.read_sql_query("SELECT * FROM players_teams", con)
players_df = pd.read_sql_query("SELECT * FROM players", con)
series_post_df = pd.read_sql_query("SELECT * FROM series_post", con)
teams_df = pd.read_sql_query("SELECT * FROM teams", con)

awards_players_df = awards_players_df.tail(-1)
coaches_df = coaches_df.tail(-1)
players_teams_df = players_teams_df.tail(-1)
players_df = players_df.tail(-1)
series_post_df = series_post_df.tail(-1)
teams_df = teams_df.tail(-1)


#teams_df = teams_df.drop(['lgID', 'rank','seeded','firstRound','semis', 'finals','GP','arena','name','tmORB','tmDRB','tmTRB','opptmORB','opptmDRB','opptmTRB','divID'], axis=1)
#teams_df = pd.get_dummies(teams_df, columns=['tmID', 'franchID', 'confID'])

teams_df_clean = teams_df[['year', 'tmID', 'confID', 'attend']]

for index, row in coaches_df.iterrows():
    year = row['year']
    tmID = row['tmID']
    coachID = row['coachID']
    won = row['won']
    lost = row['lost']

    matching_row = teams_df[(teams_df['year'] == year) & (teams_df['tmID'] == tmID)]
    if not matching_row.empty:
        teams_df.loc[matching_row.index, 'coachID'] = coachID
        teams_df.loc[matching_row.index, 'coachWins'] = won
        teams_df.loc[matching_row.index, 'coachLosses'] = lost


# players_teams_df.head()
players_teams_df = players_teams_df.drop(['year', 'stint', 'tmID', 'lgID'], axis = 1)
#players_df = 

for index, row in players_df.iterrows():
    players_df['seasonNumber'] = len(players_teams_df.loc[players_teams_df['playerID'] == row['bioId']])
    for column in players_teams_df:
        if(column != 'playerID'):
            result = players_teams_df.loc[players_teams_df['playerID'] == row['bioId'], column].mean()
            if(pd.isna(result)):
                players_df.loc[index, column] = 0
            else:
                players_df.loc[index, column] = result

players_df.to_csv('result.csv', index=False)


players_df.head()






,bioId,pos,firstseason,lastseason,height,weight,college,collegeOther,birthDate,deathDate,...,PostBlocks,PostTurnovers,PostPF,PostfgAttempted,PostfgMade,PostftAttempted,PostftMade,PostthreeAttempted,PostthreeMade,PostDQ
1,abrahta01w,C,0,0,74.0,190,George Washington,,1975-09-27,0000-00-00,...,0.000,0.00,0.000,0.000,0.000,0.0,0.0,0.0,0.000,0.0
2,abrossv01w,F,0,0,74.0,169,Connecticut,,1980-07-09,0000-00-00,...,0.375,1.75,2.875,8.625,3.125,2.0,1.5,2.5,0.625,0.0
3,adairje01w,C,0,0,76.0,197,George Washington,,1986-12-19,0000-00-00,...,0.000,0.00,0.000,0.000,0.000,0.0,0.0,0.0,0.000,0.0
4,adamsda01w,F-C,0,0,73.0,239,Texas A&M,Jefferson College (JC),1989-02-19,0000-00-00,...,0.000,0.00,0.000,0.000,0.000,0.0,0.0,0.0,0.000,0.0
5,adamsjo01w,C,0,0,75.0,180,New Mexico,,1981-05-24,0000-00-00,...,0.000,0.00,0.000,0.000,0.000,0.0,0.0,0.0,0.000,0.0


In [47]:
teams_df["playoff"] = teams_df["playoff"].map({"N": 0, "Y": 1})

X = teams_df.drop('playoff', axis=1)
y = teams_df['playoff']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy: .2f}")

print(classification_report(y_test, y_pred))

tree_rules = export_text(clf, feature_names=list(X.columns))
print("Decision Tree Rules:")
print(tree_rules)


ValueError: could not convert string to float: 'WNBA'